In [490]:
import pandas as pd

In [491]:
violations_df = pd.read_csv("violations.csv", nrows=100)

In [492]:
# 1. I want to make sure my Plate ID is a string. Can't lose the leading zeroes!

#3. I want the dates to be dates! Read the read_csv documentation to find out how to make pandas automatically parse dates.
col_types = { 'Plate ID': 'str'}
dof_violations_df = pd.read_csv("DOF_Parking_Violation_Codes.csv", dtype=col_types, parse_dates=True, nrows=100)

In [493]:
# print(violations_df.columns)

In [494]:
# len(violations_df)

In [495]:
# print(dof_violations_df.head())

2. I don't think anyone's car was built in 0AD. Discard the '0's as NaN.

In [496]:
violations_df = violations_df[violations_df['Vehicle Year'] != 0]
# print(violations_df['Vehicle Year'].head())

4. "Date first observed" is a pretty weird column, but it seems like it has a date hiding inside. Using a function with .apply, transform the string (e.g. "20140324") into a Python date. Make the 0's show up as NaN.


In [497]:
# print(violations_df['Date First Observed'].head())

In [498]:
import datetime
violations_df.head()['Issue Date'].astype(datetime.datetime)

0    08/04/2013
1    08/04/2013
3    08/05/2013
4    08/08/2013
6    08/11/2013
Name: Issue Date, dtype: object

In [499]:
import datetime
violations_df.head()['Issue Date'].astype(datetime.datetime)

#violations_df['Issue Date'] = violations_df['Issue Date'].astype('datetime64[ns]')


0    08/04/2013
1    08/04/2013
3    08/05/2013
4    08/08/2013
6    08/11/2013
Name: Issue Date, dtype: object

In [500]:
# violations_df.dtypes

5. "Violation time" is... not a time. Make it a time.

In [501]:
import re
# violations_df['Violation Time'].head()

In [502]:
import numpy as np
# Build a function using that method
def time_to_datetime(str_time):
    try:
    #str_time = re.sub('^0', '', str_time)
        if isinstance(str_time, str):
            str_time = str_time + "M"
            #print("Trying to convert", str_time, "into a time")
            return datetime.datetime.strptime(str_time.strip(), "%I%M%p").time()
    #try:
     #   if str_time == -999:
      #      print("It's -999")
       #     return np.nan
    
    except:
        return np.nan

In [503]:

# Apply that method to the 'Time' column of the dataframe
violations_df['Violation Time'] = violations_df['Violation Time'].apply(time_to_datetime)

In [504]:
print(violations_df['Violation Time'].head())

0    07:52:00
1    12:40:00
3    14:32:00
4    12:39:00
6    19:41:00
Name: Violation Time, dtype: object


In [505]:
# def remove_1900(year):
#     year = str(year)
#     year = re.sub('^1900-01-01', '', year)
#     return year

# print(violations_df['Violation Time'].apply(remove_1900))

6.  There sure are a lot of colors of cars, too bad so many of them are the same. Make "BLK" and "BLACK", "WT" and "WHITE", and any other combinations that you notice.

In [506]:
import re

In [507]:
def vehicle_colors(vehicle):
    if isinstance(vehicle, str):
        #print(vehicle)
        vehicle = re.sub('^GY$', 'GREY', vehicle)
        #vehicle = vehicle.replace("GY", "GREY")
        vehicle = re.sub('^WH$', 'WHITE', vehicle)
        vehicle = re.sub('^BR$', 'BROWN', vehicle)
        vehicle = re.sub('^RD$', 'RED', vehicle)
        vehicle = re.sub('^B[LK]$', 'BLACK', vehicle)
        vehicle = re.sub('^TN$', 'TAN', vehicle)
        vehicle = re.sub('^YW$', 'YELLOW', vehicle)
        vehicle = re.sub('^SIL$', 'SILVER', vehicle)
        vehicle = re.sub('^GR$', 'GREEN', vehicle)
        vehicle = re.sub('^SILVE$', 'SILVER', vehicle)
       
       
        
#         vehicle = vehicle.replace("WH","WHITE")
#         vehicle = vehicle.replace("BR","BROWN")
#         vehicle = vehicle.replace("RD","RED")
#         vehicle = vehicle.replace("BL","BLACK")
#         vehicle = vehicle.replace("BK","BLACK")
#         vehicle = vehicle.replace("TN","TAN")
#         vehicle = vehicle.replace("YW","YELLOW")
#         vehicle = re.sub('SIL$', 'SILVER', vehicle)
#         vehicle = vehicle.replace("SILVR","SILVER")
#         vehicle = vehicle.replace("SIL","SILVER")
    return vehicle

violations_df['Vehicle Color'] = violations_df['Vehicle Color'].apply(vehicle_colors)

In [508]:
violations_df['Vehicle Color'].head()

0     GREY
1    WHITE
3    WHITE
4    BROWN
6       GN
Name: Vehicle Color, dtype: object

7. Join the data with the Parking Violations Code dataset from the NYC Open Data site.

In [509]:
print(violations_df.columns)
violations_df['Violation Code'].head()

Index(['Summons Number', 'Plate ID', 'Registration State', 'Plate Type',
       'Issue Date', 'Violation Code', 'Vehicle Body Type', 'Vehicle Make',
       'Issuing Agency', 'Street Code1', 'Street Code2', 'Street Code3',
       'Vehicle Expiration Date', 'Violation Location', 'Violation Precinct',
       'Issuer Precinct', 'Issuer Code', 'Issuer Command', 'Issuer Squad',
       'Violation Time', 'Time First Observed', 'Violation County',
       'Violation In Front Of Or Opposite', 'House Number', 'Street Name',
       'Intersecting Street', 'Date First Observed', 'Law Section',
       'Sub Division', 'Violation Legal Code', 'Days Parking In Effect    ',
       'From Hours In Effect', 'To Hours In Effect', 'Vehicle Color',
       'Unregistered Vehicle?', 'Vehicle Year', 'Meter Number',
       'Feet From Curb', 'Violation Post Code', 'Violation Description',
       'No Standing or Stopping Violation', 'Hydrant Violation',
       'Double Parking Violation'],
      dtype='object')


0    46
1    46
3    46
4    41
6    14
Name: Violation Code, dtype: int64

In [510]:
violations_df

,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,...,Vehicle Color,Unregistered Vehicle?,Vehicle Year,Meter Number,Feet From Curb,Violation Post Code,Violation Description,No Standing or Stopping Violation,Hydrant Violation,Double Parking Violation
0,1283294138,GBB9093,NY,PAS,08/04/2013,46,SUBN,AUDI,P,37250,...,GREY,0,2013,-,0,NaN,NaN,NaN,NaN,NaN
1,1283294151,62416MB,NY,COM,08/04/2013,46,VAN,FORD,P,37290,...,WHITE,0,2012,-,0,NaN,NaN,NaN,NaN,NaN
3,1283294175,63009MA,NY,COM,08/05/2013,46,VAN,FORD,P,37270,...,WHITE,0,2010,-,0,NaN,NaN,NaN,NaN,NaN
4,1283294187,91648MC,NY,COM,08/08/2013,41,TRLR,GMC,P,37240,...,BROWN,0,2012,-,0,NaN,NaN,NaN,NaN,NaN
6,1283294229,GCR2838,NY,PAS,08/11/2013,14,VAN,NaN,P,37250,...,GN,0,2011,-,0,NaN,NaN,NaN,NaN,NaN
8,1283983631,GBH9379,NY,PAS,08/07/2013,24,SDN,TOYOT,X,63430,...,WHITE,0,2001,-,0,NaN,NaN,NaN,NaN,NaN
9,1283983667,MCL78B,NJ,PAS,07/18/2013,24,SDN,SUBAR,H,0,...,BLACK,0,2005,-,0,NaN,NaN,NaN,NaN,NaN
10,1283983679,M367CN,NY,PAS,07/18/2013,24,SDN,HYUND,H,0,...,BLUE,0,2010,-,0,NaN,NaN,NaN,NaN,NaN
11,1283983734,GAR6813,NY,PAS,07/18/2013,24,SDN,TOYOT,H,0,...,TAN,0,1998,-,0,NaN,NaN,NaN,NaN,NaN
13,1283983825,GAC2703,NY,PAS,08/12/2013,24,SDN,NISSA,X,23230,...,WHITE,0,2012,-,0,NaN,NaN,NaN,NaN,NaN


In [511]:
print(dof_violations_df.columns)
dof_violations_df['CODE'].head()

Index(['CODE', 'DEFINITION', 'Manhattan  96th St. & below', 'All Other Areas'], dtype='object')


0    10
1    11
2    12
3    13
4    14
Name: CODE, dtype: object

In [512]:
# violations_df.merge(dof_violations_df, left_on='Violation Code', right_on='CODE')

In [513]:
#print(dof_violations_df['CODE'][0])
#test = dof_violations_df['CODE'][0]

def to_int(test):
    try:
        # test = re.sub('$', '', test)
        test = test.strip("$")
        #print(test)
        test = int(test)
        #print(test, " is now an int")
        return test
    except:
        print(test, " coult not be converted to an int")
        return np.nan

In [514]:
dof_violations_df['CODE'] = dof_violations_df['CODE'].apply(to_int)

37-38  coult not be converted to an int


7. Join the data with the Parking Violations Code dataset from the NYC Open Data site.

In [515]:
violations_df = violations_df.merge(dof_violations_df, left_on='Violation Code', right_on='CODE')

8. How much money did NYC make off of parking violations?

In [516]:
violations_df.describe()

/Users/Monica/.virtualenvs/dataanalysis/lib/python3.5/site-packages/numpy/lib/function_base.py:3823: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,Summons Number,Violation Code,Street Code1,Street Code2,Street Code3,Vehicle Expiration Date,Violation Location,Violation Precinct,Issuer Precinct,Issuer Code,...,Violation Legal Code,Unregistered Vehicle?,Vehicle Year,Feet From Curb,Violation Post Code,Violation Description,No Standing or Stopping Violation,Hydrant Violation,Double Parking Violation,CODE
count,4.400000e+01,44.000000,44.000000,44.000000,44.000000,4.400000e+01,43.000000,44.000000,44.000000,44.000000,...,0.0,44.0,44.000000,44.000000,0.0,0.0,0.0,0.0,0.0,44.000000
mean,1.289335e+09,30.113636,31847.272727,28144.340909,28321.454545,1.876975e+07,61.674419,60.272727,279.386364,679245.500000,...,NaN,0.0,2007.090909,0.227273,NaN,NaN,NaN,NaN,NaN,30.113636
std,4.022321e+06,18.104626,25830.368405,20609.272005,20557.940516,5.135935e+06,39.000440,39.649842,348.814018,338298.083452,...,NaN,0.0,5.753940,1.361662,NaN,NaN,NaN,NaN,NaN,18.104626
min,1.283294e+09,14.000000,0.000000,0.000000,0.000000,0.000000e+00,1.000000,0.000000,0.000000,0.000000,...,NaN,0.0,1995.000000,0.000000,NaN,NaN,NaN,NaN,NaN,14.000000
25%,1.286216e+09,17.000000,14505.000000,13210.000000,15578.750000,2.014066e+07,NaN,20.000000,0.000000,343494.500000,...,NaN,0.0,2002.000000,0.000000,NaN,NaN,NaN,NaN,NaN,17.000000
50%,1.290054e+09,24.000000,27615.000000,26965.000000,25680.000000,2.014093e+07,NaN,73.000000,106.000000,919856.000000,...,NaN,0.0,2010.000000,0.000000,NaN,NaN,NaN,NaN,NaN,24.000000
75%,1.293096e+09,41.500000,44675.000000,40404.000000,39870.750000,2.014345e+07,NaN,85.000000,458.500000,930130.000000,...,NaN,0.0,2012.000000,0.000000,NaN,NaN,NaN,NaN,NaN,41.500000
max,1.294713e+09,98.000000,93230.000000,74830.000000,83330.000000,2.015102e+07,120.000000,120.000000,976.000000,947582.000000,...,NaN,0.0,2013.000000,9.000000,NaN,NaN,NaN,NaN,NaN,98.000000


In [517]:
violations_df.columns

Index(['Summons Number', 'Plate ID', 'Registration State', 'Plate Type',
       'Issue Date', 'Violation Code', 'Vehicle Body Type', 'Vehicle Make',
       'Issuing Agency', 'Street Code1', 'Street Code2', 'Street Code3',
       'Vehicle Expiration Date', 'Violation Location', 'Violation Precinct',
       'Issuer Precinct', 'Issuer Code', 'Issuer Command', 'Issuer Squad',
       'Violation Time', 'Time First Observed', 'Violation County',
       'Violation In Front Of Or Opposite', 'House Number', 'Street Name',
       'Intersecting Street', 'Date First Observed', 'Law Section',
       'Sub Division', 'Violation Legal Code', 'Days Parking In Effect    ',
       'From Hours In Effect', 'To Hours In Effect', 'Vehicle Color',
       'Unregistered Vehicle?', 'Vehicle Year', 'Meter Number',
       'Feet From Curb', 'Violation Post Code', 'Violation Description',
       'No Standing or Stopping Violation', 'Hydrant Violation',
       'Double Parking Violation', 'CODE', 'DEFINITION',
       '

In [518]:
violations_df['Summons Number']

0     1283294138
1     1283294151
2     1283294175
3     1290834349
4     1293030739
5     1293100924
6     1293525625
7     1283294229
8     1286036800
9     1286246398
10    1286248000
11    1286807633
12    1291898384
13    1293101783
14    1293107761
15    1293525560
16    1283983631
17    1283983667
18    1283983679
19    1283983734
20    1283983825
21    1286123550
22    1286289841
23    1286799648
24    1286807475
25    1287320491
26    1289971754
27    1291896399
28    1293090530
29    1287320570
30    1288135920
31    1292755052
32    1292756792
33    1288259270
34    1290136210
35    1290834362
36    1291905236
37    1293101758
38    1294712779
39    1294712792
40    1294712901
41    1294712950
42    1293094389
43    1294712755
Name: Summons Number, dtype: int64

9. What's the most lucrative kind of parking violation? The most frequent?
10. New Jersey has bad drivers, but does it have bad parkers, too? How much money does NYC make off of all non-New York vehicles?
11. Make a chart of the top few.
12. What time of day do people usually get their tickets? You can break the day up into several blocks - for example 12am-6am, 6am-12pm, 12pm-6pm, 6pm-12am.
13. What's the average ticket cost in NYC?
14. Make a graph of the number of tickets per day.
15. Make a graph of the amount of revenue collected per day.
16. Manually construct a dataframe out of https://dmv.ny.gov/statistic/2015licinforce-web.pdf (only NYC boroughts - bronx, queens, manhattan, staten island, brooklyn), having columns for borough name, abbreviation, and number of licensed drivers.
17. What's the parking-ticket-$-per-licensed-driver in each borough of NYC? Do this with pandas and the dataframe you just made, not with your head!

In [519]:
violations_df['Manhattan\xa0 96th St. & below']

0     $115 
1     $115 
2     $115 
3     $115 
4     $115 
5     $115 
6     $115 
7     $115 
8     $115 
9     $115 
10    $115 
11    $115 
12    $115 
13    $115 
14    $115 
15    $115 
16     $65 
17     $65 
18     $65 
19     $65 
20     $65 
21     $65 
22    $115 
23    $115 
24    $115 
25    $115 
26    $115 
27    $115 
28    $115 
29    $115 
30    $115 
31    $115 
32    $115 
33    $115 
34     $65 
35     $95 
36     $95 
37     $95 
38     $95 
39     $95 
40     $95 
41     $95 
42    $115 
43     $95 
Name: Manhattan  96th St. & below, dtype: object

In [520]:
violations_df['Manhattan\xa0 96th St. & below'] = violations_df['Manhattan\xa0 96th St. & below'].apply(to_int)

In [521]:
violations_df['Manhattan\xa0 96th St. & below'].sum()

4550

In [522]:
violations_df['All Other Areas'] = violations_df['All Other Areas'].apply(to_int)

In [523]:
violations_df['All Other Areas'].sum()

4520

In [524]:
violations_df['Manhattan\xa0 96th St. & below'].describe()

count     44.000000
mean     103.409091
std       18.545128
min       65.000000
25%       95.000000
50%      115.000000
75%      115.000000
max      115.000000
Name: Manhattan  96th St. & below, dtype: float64

In [525]:
def oneonefive(violation):
    if violation['Manhattan\xa0 96th St. & below'] == 115:
        print(violation)
violations_df.apply(oneonefive)

KeyError: ('Manhattan\xa0 96th St. & below', 'occurred at index Summons Number')

In [ ]:
violations_df